# CCS potential with LAMMPS  for ZnO

In this example, we will run CCS potentials with LAMMPS for multicomponent systems. ZnO  CCS potential is used for this illustration. Note that the exponential head part in **CCS_params.json** is ignored when LAMMPS spline tables are created. The exponential part is avoided because LAMMPS performs a pre-interpolation on the tabular data, and often pre-interpolation done with exponential head gives worse results. In general, a higher resolution should be used while generating spline table for LAMMPS. By default, the gridsize used for CCS is divided by 10.  

In [1]:
# Generates CCS.table file readable by LAMMPS
from ccs_fit.scripts.jsonTotable import asecalcTotable
tags=asecalcTotable("CCS_params.json",scale=10)  # Controls the resolution of the gridsize; gridsize=dr/scale.
tags

{'O-Zn': {'Rmin': 1.2999991089415996,
  'Rcut': 5.9999991089416,
  'dr': 0.005,
  'N': 941},
 'O-O': {'Rmin': 2.0999990640749275,
  'Rcut': 5.9999990640749274,
  'dr': 0.01,
  'N': 391},
 'Zn-Zn': {'Rmin': 1.9999957089651335,
  'Rcut': 5.999995708965134,
  'dr': 0.01,
  'N': 401}}

In [2]:
!head CCS.table


 O-Zn
 N 941 R 1.2999991089415996 5.9999991089416 

 1 1.2999991089415996 18.6733869381274 391.5424306774645
 2 1.3049991089415995 16.74203855060834 380.99692432954544
 3 1.3099991089415994 14.863417694838336 370.45141797784134
 4 1.3149991089415993 13.037524370836312 359.9059116223522
 5 1.3199991089415992 11.264358578621199 349.3604052630781
 6 1.3249991089415991 9.543920318211915 338.814898900019


In [3]:
import json
import copy
import matplotlib.pyplot as plt

from ase import Atoms
from ase.calculators.lammpsrun import LAMMPS
from ccs_fit.ase_calculator.ccs_ase_calculator import CCS
from ase.io import read,write
from ase.io.trajectory import Trajectory
from ase.optimize import BFGS
from sklearn.metrics import mean_squared_error


with open ('CCS_params.json','r') as f:
    CCS_params=json.load(f)


# Parameters for LAMMPS 
parameters = {
              'pair_style': 'table spline 1000',   # The number of elements chosen taken for pre-interpolation
              'pair_coeff': ['1 1  CCS.table O-O 5.9999990640749274', # Specify Rcut for each pair
                             '1 2  CCS.table O-Zn 5.9999991089416',
                             '2 2  CCS.table Zn-Zn 5.999995708965134'],
#              'pair_write': ['1 2 500 r  1.2999991089415996 5.9999991089416 table.txt table']
              'command': '/usr/bin/lmp'   # Remove or change to your local lammps executable
}



# Single point evaluation

In [4]:
struct=read('POSCAR')
lammps = LAMMPS(parameters=parameters,keep_tmp_files=False)
ccs= CCS(CCS_params)


struct_ase=copy.deepcopy(struct)

struct.calc = lammps
print("Energy from LAMMPS  pre optmization: ", struct.get_potential_energy())
struct_ase.calc=ccs
print("Energy from CCS calculator pre optmization:", struct_ase.get_potential_energy())

print ("MSE on energy between ase calc and LAMMPS: {}".format((struct_ase.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between ase calc and LAMMPS: {}".format(mean_squared_error(struct.get_forces(),struct_ase.get_forces())))




/home/thism292/anaconda3/envs/CCS_fit/lib/python3.9/site-packages/ase/calculators/lammpsrun.py:191: UserWarning: You are using an old syntax to set 'parameters'.
Please use LAMMPSRUN.set().
  warnings.warn(self.legacy_warn_string.format("parameters"))


Energy from LAMMPS  pre optmization:  -519.1112078503346
Energy from CCS calculator pre optmization: -519.1112078626454
MSE on energy between ase calc and LAMMPS: 1.5155595905299652e-16
MSE on forces between ase calc and LAMMPS: 2.905083127909829e-11


/home/thism292/anaconda3/envs/CCS_fit/lib/python3.9/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.rank instead
  warnings.warn(warning)


# Optimization

In [5]:
traj = Trajectory('LAMMPS.traj', 'w', struct)
dyn=BFGS(struct)
dyn.attach(traj)
dyn.run(fmax=0.05)
print("Energy from LAMMPS after optimization: ", struct.get_potential_energy())


traj = Trajectory('CCS.traj', 'w', struct_ase)
dyn=BFGS(struct_ase)
dyn.attach(traj)
dyn.run(fmax=0.05)
print("Energy from CCS calculator after optimization:", struct_ase.get_potential_energy())

print ("MSE on energy between ase calc and LAMMPS: {}".format((struct_ase.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between ase calc and LAMMPS: {}".format(mean_squared_error(struct.get_forces(),struct_ase.get_forces())))



      Step     Time          Energy         fmax
BFGS:    0 09:39:09     -519.111208        2.4049
BFGS:    1 09:39:09     -519.861091        2.3442
BFGS:    2 09:39:09     -522.739661        4.2708
BFGS:    3 09:39:09     -523.828807        4.4097
BFGS:    4 09:39:09     -525.336034        4.3607
BFGS:    5 09:39:09     -527.125758        4.2525
BFGS:    6 09:39:09     -527.586885        6.0038
BFGS:    7 09:39:09     -529.373394        1.8725
BFGS:    8 09:39:09     -530.112725        1.9523
BFGS:    9 09:39:09     -531.230579        2.7450
BFGS:   10 09:39:09     -531.494644        1.7438
BFGS:   11 09:39:09     -531.546575        1.6485
BFGS:   12 09:39:09     -531.650559        0.3163
BFGS:   13 09:39:09     -531.654912        0.1985
BFGS:   14 09:39:09     -531.656742        0.0670
BFGS:   15 09:39:09     -531.656984        0.0341
Energy from LAMMPS after optimization:  -531.6569841018262
      Step     Time          Energy         fmax
BFGS:    0 09:39:09     -519.111208        

/home/thism292/anaconda3/envs/CCS_fit/lib/python3.9/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.rank instead
  warnings.warn(warning)


BFGS:    8 09:39:09     -530.112682        1.9521
BFGS:    9 09:39:09     -531.230655        2.7446
BFGS:   10 09:39:09     -531.494997        1.7433
BFGS:   11 09:39:09     -531.546192        1.6499
BFGS:   12 09:39:09     -531.650564        0.3167
BFGS:   13 09:39:09     -531.654920        0.1990
BFGS:   14 09:39:09     -531.656755        0.0669
BFGS:   15 09:39:09     -531.656999        0.0341
Energy from CCS calculator after optimization: -531.6569988000888
MSE on energy between ase calc and LAMMPS: 2.1603892314293773e-10
MSE on forces between ase calc and LAMMPS: 2.385570490930619e-09


**Note** : 
* Onebody energy contributons in CCS_params file was set to 0 for easy comparison